In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import time
import copy

# IMPORT YOUR UNIVERSAL PREPROCESSING MODULE
from data_preprocessing import get_dataloaders

# ==========================================
# 1. CONFIGURATION
# ==========================================
BATCH_SIZE = 32
NUM_CLASSES = 196
EPOCHS = 25
LEARNING_RATE = 0.0001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_SAVE_PATH = "resnet50_new_hugg_preproce.pth"

# ==========================================
# 2. MODEL ARCHITECTURE
# ==========================================
def get_resnet_model(num_classes):
    print("🏗️ Initializing ResNet-50...")
    # Load Pre-trained Weights (ImageNet)
    model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
    
    # Freeze initial layers to retain basic features
    for param in model.parameters():
        param.requires_grad = False
        
    # Unfreeze deep layers (Layer 3 and 4) for fine-grained adaptation
    for param in model.layer3.parameters():
        param.requires_grad = True
    for param in model.layer4.parameters():
        param.requires_grad = True
        
    # Custom Classification Head
    # Replacing the final fully connected layer
    in_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(in_features, 1024),
        nn.ReLU(),
        nn.Dropout(0.5), # Strong dropout to prevent overfitting
        nn.Linear(1024, num_classes)
    )
    return model

# ==========================================
# 3. TRAINING FUNCTION
# ==========================================
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=15):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    print(f"🚀 Training on device: {DEVICE}")

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                dataloader = train_loader
            else:
                model.eval()   # Set model to evaluate mode
                dataloader = val_loader

            running_loss = 0.0
            running_corrects = 0
            total_samples = 0

            # Iterate over data
            for inputs, labels in dataloader:
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                # Track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                total_samples += inputs.size(0)

            epoch_loss = running_loss / total_samples
            epoch_acc = running_corrects.double() / total_samples

            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model if it's the best one so far
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                # Save checkpoint
                torch.save(model.state_dict(), MODEL_SAVE_PATH)
                print(f"🌟 New Best Model Saved! (Acc: {best_acc:.4f})")
            
            # Step the scheduler at the end of validation
            if phase == 'val':
                scheduler.step(epoch_loss)

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best Validation Accuracy: {best_acc:.4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

# ==========================================
# 4. MAIN EXECUTION
# ==========================================
if __name__ == "__main__":
    # 1. Load Data using the Universal Preprocessing Module
    train_dl, val_dl, test_dl = get_dataloaders(BATCH_SIZE)
    
    # 2. Initialize Model
    resnet_model = get_resnet_model(NUM_CLASSES)
    resnet_model = resnet_model.to(DEVICE)
    
    # 3. Setup Training Configuration
    criterion = nn.CrossEntropyLoss()
    
    # Use Adam optimizer with weight decay for regularization
    optimizer = optim.Adam(
        resnet_model.parameters(), 
        lr=LEARNING_RATE, 
        weight_decay=1e-4
    )
    
    # Learning Rate Scheduler (Reduce LR when validation loss plateaus)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.1, patience=3, verbose=True
    )
    
    # 4. Start Training
    try:
        trained_model = train_model(
            resnet_model, 
            train_dl, 
            val_dl, 
            criterion, 
            optimizer, 
            scheduler, 
            num_epochs=EPOCHS
        )
        print("✅ Process Completed Successfully.")
        
    except KeyboardInterrupt:
        print("\n🛑 Training interrupted by user.")
    except Exception as e:
        print(f"\n❌ An error occurred: {e}")

 Loading 'tanganke/stanford_cars' from Hugging Face Hub...
✅ Data Split: 6515 Train | 1629 Val | 8041 Test
🏗️ Initializing ResNet-50...
🚀 Training on device: cuda
Epoch 1/25
----------
Train Loss: 5.0178 Acc: 0.0301
Val Loss: 4.3781 Acc: 0.0890
🌟 New Best Model Saved! (Acc: 0.0890)

Epoch 2/25
----------
Train Loss: 3.8941 Acc: 0.1199
Val Loss: 3.1494 Acc: 0.2124
🌟 New Best Model Saved! (Acc: 0.2124)

Epoch 3/25
----------
Train Loss: 2.9248 Acc: 0.2503
Val Loss: 2.2456 Acc: 0.4211
🌟 New Best Model Saved! (Acc: 0.4211)

Epoch 4/25
----------
Train Loss: 2.1993 Acc: 0.3840
Val Loss: 1.8091 Acc: 0.4972
🌟 New Best Model Saved! (Acc: 0.4972)

Epoch 5/25
----------
Train Loss: 1.7017 Acc: 0.5085
Val Loss: 1.4480 Acc: 0.5881
🌟 New Best Model Saved! (Acc: 0.5881)

Epoch 6/25
----------
Train Loss: 1.2954 Acc: 0.6138
Val Loss: 1.1901 Acc: 0.6599
🌟 New Best Model Saved! (Acc: 0.6599)

Epoch 7/25
----------
Train Loss: 1.0000 Acc: 0.6976
Val Loss: 0.9720 Acc: 0.7207
🌟 New Best Model Saved! (Acc: